In [ ]:
import caffe
import cv2
import numpy as np
import os
import utils


def get_neuron_diffs(net, cur_l, pos_l, img_original, img_mutated):
    net.blobs['data'].data[...] = utils.get_data(img_original)
    net.forward(end=pos_l)
    data_original = net.blobs[cur_l].data[0].copy()

    net.blobs['data'].data[...] = utils.get_data(img_mutated)
    net.forward(end=pos_l)
    data_mutated = net.blobs[cur_l].data[0].copy()

    diffs = []
    for i in range(len(data_original)):
        diffs.append(np.sum(abs(data_mutated[i] - data_original[i])))

    return diffs

In [ ]:
caffe.set_mode_cpu()
vgg_root   = '../data/vgg_face_caffe/'
vgg_deploy = vgg_root + 'VGG_FACE_deploy.prototxt'
vgg_weight = vgg_root + 'VGG_FACE.caffemodel'
vgg_net    = caffe.Net(vgg_deploy, vgg_weight, caffe.TEST)
vgg_layers = utils.get_layers(vgg_net)

In [ ]:
attributes = ['leye', 'reye', 'nose', 'mouth']
actions    = ['substitution', 'preservation']
root_path  = '../data/attribute_mutated/'
base_img   = '../data/base_img.jpg'

attri = attributes[0]

f = open('../data/witnesses/' + attri + '_witness.txt', 'w')

for idx in range(1, len(vgg_layers)-2):
    cur_l = vgg_layers[idx]
    pos_l = vgg_layers[idx+1]

    if 'pool' in cur_l:
        layer_size = utils.get_layer_size(vgg_net, vgg_layers[idx-1])
    else:
        layer_size = utils.get_layer_size(vgg_net, cur_l)

    neuron_set = set(range(layer_size))

    for img_name in os.listdir(root_path + attri + '_' + actions[0]):
        attri_sub = root_path + attri + '_' + actions[0] + '/' + img_name
        attri_pre = root_path + attri + '_' + actions[1] + '/' + img_name

        diff_sub = get_neuron_diffs(vgg_net, cur_l, pos_l, base_img, attri_sub)
        diff_pre = get_neuron_diffs(vgg_net, cur_l, pos_l, base_img, attri_pre)

        sub_set = set([i for i in range(len(diff_sub)) if diff_sub[i] > np.median(diff_sub)])
        neuron_set.intersection_update(sub_set)

        pre_set = set([i for i in range(len(diff_pre)) if diff_pre[i] <= np.median(diff_pre)])
        neuron_set.intersection_update(pre_set)

        if not neuron_set:
            break

    if len(neuron_set) != 0:
        print('{}\t: {}'.format(cur_l, list(neuron_set)))
        f.write(cur_l + ',' + str(list(neuron_set))[1:-1] + '\n')
    else:
        print('{}\t: {}'.format(cur_l, 'None'))

f.close()